# Setup

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install accelerate -U
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.9 MB/s eta 0:00:00


In [ ]:
from transformers import file_utils
print(file_utils.default_cache_path)

/root/.cache/huggingface/hub


In [ ]:


!ls /root/.cache/huggingface/hub/



models--decapoda-research--llama-13b-hf  tmpm7zr4vs8  tmpsqlm86mi  tmpwfqhimid
tmp295klh7g				 tmpnpmnh996  tmpsu6n8zq9  tmpzd9sfgy2
tmp4x03vmlk				 tmprceq3_7r  tmpv4pq7am7  version.txt


In [ ]:
# Download base and delta

from huggingface_hub import snapshot_download
checkpoint_delta = "IDEA-CCNL/Ziya-LLaMA-13B-v1"
checkpoint_base = "decapoda-research/llama-13b-hf"

snapshot_download(repo_id=checkpoint_base)
# snapshot_download(repo_id=checkpoint_delta)

Fetching 51 files:   0%|          | 0/51 [00:00<?, ?it/s]

'/root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce'

In [ ]:
!wget https://github.com/IDEA-CCNL/Fengshenbang-LM/blob/main/fengshen/utils/apply_delta.py .

--2023-07-11 01:37:30--  https://github.com/IDEA-CCNL/Fengshenbang-LM/blob/main/fengshen/utils/apply_delta.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36904 (36K) [text/plain]
Saving to: ‘apply_delta.py’

apply_delta.py      100%[===================>]  36.04K  --.-KB/s    in 0.01s   

2023-07-11 01:37:31 (2.99 MB/s) - ‘apply_delta.py’ saved [36904/36904]

--2023-07-11 01:37:31--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2023-07-11 01:37:31--
Total wall clock time: 0.4s
Downloaded: 1 files, 36K in 0.01s (2.99 MB/s)


# Define paths

In [ ]:
base_model_path = "/root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce/"
delta_path = "/root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya-LLaMA-13B-v1/snapshots/fccf34387d2c9f2f95ff59ae380e6de3718e41ff"
target_model_path = '/content/target/tmp/'

In [ ]:
base_model_path=base_model_path
target_model_path=target_model_path
delta_path=delta_path













# Original merging code

In [ ]:
import argparse
import gc
import glob
import json
import os
import shutil
import tempfile

from huggingface_hub import snapshot_download
import torch
from torch import nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, LlamaForCausalLM


GB = 1 << 30


def split_files(model_path, tmp_path, split_size):
    if not os.path.exists(model_path):
        model_path = snapshot_download(repo_id=model_path)
    if not os.path.exists(tmp_path):
        os.makedirs(tmp_path)

    file_pattern = os.path.join(model_path, "pytorch_model-*.bin")
    files = glob.glob(file_pattern)

    part = 0
    try:
        for file_path in tqdm(files):
            state_dict = torch.load(file_path)
            new_state_dict = {}

            current_size = 0
            for name, param in state_dict.items():
                param_size = param.numel() * param.element_size()

                if current_size + param_size > split_size:
                    new_file_name = f"pytorch_model-{part}.bin"
                    new_file_path = os.path.join(tmp_path, new_file_name)
                    torch.save(new_state_dict, new_file_path)
                    current_size = 0
                    new_state_dict = None
                    gc.collect()
                    new_state_dict = {}
                    part += 1

                new_state_dict[name] = param
                current_size += param_size

            new_file_name = f"pytorch_model-{part}.bin"
            new_file_path = os.path.join(tmp_path, new_file_name)
            torch.save(new_state_dict, new_file_path)
            new_state_dict = None
            gc.collect()
            new_state_dict = {}
            part += 1
    except Exception as e:
        print(f"An error occurred during split_files: {e}")
        shutil.rmtree(tmp_path)
        raise


def apply_delta_low_cpu_mem(base_model_path, target_model_path, delta_path):
    delta_tokenizer = AutoTokenizer.from_pretrained(delta_path, use_fast=False)
    delta_config = AutoConfig.from_pretrained(delta_path)

    if os.path.exists(target_model_path):
        shutil.rmtree(target_model_path)
    os.makedirs(target_model_path)

    split_size = 4 * GB

    with tempfile.TemporaryDirectory() as tmp_base_path, tempfile.TemporaryDirectory() as tmp_delta_path:
        print(f"Split files for the base model to {tmp_base_path}")
        split_files(base_model_path, tmp_base_path, split_size)
        print(f"Split files for the delta weights to {tmp_delta_path}")
        split_files(delta_path, tmp_delta_path, split_size)

        base_pattern = os.path.join(tmp_base_path, "pytorch_model-*.bin")
        base_files = glob.glob(base_pattern)
        delta_pattern = os.path.join(tmp_delta_path, "pytorch_model-*.bin")
        delta_files = glob.glob(delta_pattern)
        delta_state_dict = torch.load(delta_files[0])

        print("Applying the delta")
        weight_map = {}
        total_size = 0

        for i, base_file in tqdm(enumerate(base_files)):
            state_dict = torch.load(base_file)
            file_name = f"pytorch_model-{i}.bin"
            for name, param in state_dict.items():
                if name not in delta_state_dict:
                    for delta_file in delta_files:
                        delta_state_dict = torch.load(delta_file)
                        gc.collect()
                        if name in delta_state_dict:
                            break
                if "embed_tokens" in name or "lm_head.weight" in name or "self_attn.rotary_emb.inv_freq" in name:
                    state_dict[name] = delta_state_dict[name]
                else:
                    state_dict[name] += delta_state_dict[name]
                weight_map[name] = file_name
                total_size += param.numel() * param.element_size()
                gc.collect()
            torch.save(state_dict, os.path.join(target_model_path, file_name))

        with open(
            os.path.join(target_model_path, "pytorch_model.bin.index.json"), "w"
        ) as f:
            json.dump(
                {"weight_map": weight_map, "metadata": {"total_size": total_size}}, f
            )

    print(f"Saving the target model to {target_model_path}")
    delta_tokenizer.save_pretrained(target_model_path)
    delta_config.save_pretrained(target_model_path)


def apply_delta(base_model_path, target_model_path, delta_path):
    print(f"Loading the delta weights from {delta_path}")
    delta_tokenizer = AutoTokenizer.from_pretrained(delta_path,     use_fast=False, local_files_only=True,)
    delta = LlamaForCausalLM.from_pretrained(
        delta_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, local_files_only=True,
    )

    print(f"Loading the base model from {base_model_path}")
    base = LlamaForCausalLM.from_pretrained(
        base_model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, local_files_only=True,
    )

    print("Applying the delta")
    for name, param in tqdm(delta.state_dict().items(), desc="Applying delta"):
        assert name in base.state_dict()
        # param.data += delta.state_dict()[name]
        if "embed_tokens" in name or "lm_head.weight" in name or "self_attn.rotary_emb.inv_freq" in name:
            continue
        else:
            param.data += base.state_dict()[name]
    # base.model.embed_tokens = delta.model.embed_tokens
    # base.lm_head.weight = delta.lm_head.weight

    print(f"Saving the target model to {target_model_path}")
    delta.save_pretrained(target_model_path)
    delta_tokenizer.save_pretrained(target_model_path)

# !python3 -m apply_delta.py --base /root/.cache/huggingface/hub/models--decapoda-research--llama-7b-hf --target /root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya-LLaMA-13B --delta /root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya-LLaMA-13B-v1
# apply_delta_low_cpu_mem(base_model_path=base_model_path,
#             target_model_path="/root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya-LLaMA-13B-v1/snapshots/tmp/",
#             delta_path=delta_path,
#             )

# apply_delta_low_cpu_mem

In [ ]:
delta_tokenizer = AutoTokenizer.from_pretrained(delta_path, use_fast=False)
delta_config = AutoConfig.from_pretrained(delta_path)

if os.path.exists(target_model_path):
    shutil.rmtree(target_model_path)
os.makedirs(target_model_path)

# split_size = 6 * GB
split_size = 4 * GB

# with tempfile.TemporaryDirectory() as tmp_base_path, tempfile.TemporaryDirectory() as tmp_delta_path:
tmp_base_path = '/content/tmp/base'
tmp_delta_path = '/content/tmp/delta'

# The file is not large. Just copy them
# !cp /root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya-LLaMA-13B-v1/snapshots/fccf34387d2c9f2f95ff59ae380e6de3718e41ff/*.bin tmp/delta/
# !cp /root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce/*.bin tmp/base/

# !rm -rf /root/.cache/huggingface/hub/models--IDEA-CCNL--Ziya-LLaMA-13B-v1/snapshots/fccf34387d2c9f2f95ff59ae380e6de3718e41ff/*.bin
# !rm -rf  /root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce/*.bin

# This is the original code to split the files which are not necessary
# print(f"Split files for the base model to {tmp_base_path}")
# split_files(base_model_path, tmp_base_path, split_size)
# print(f"Split files for the delta weights to {tmp_delta_path}")
# split_files(delta_path, tmp_delta_path, split_size)

Split files for the base model to /content/tmp/base


100%|██████████| 33/33 [03:18<00:00,  6.00s/it]


# New Section

In [ ]:
START FROM HERE
# remove some base files and do the merge. and remove and do


base_pattern = os.path.join(tmp_base_path, "pytorch_model-*.bin")
base_files = glob.glob(base_pattern)
delta_pattern = os.path.join(tmp_delta_path, "pytorch_model-*.bin")
delta_files = glob.glob(delta_pattern)
delta_state_dict = torch.load(delta_files[0])

print("Applying the delta")
weight_map = {}
total_size = 0

# # Fix the order of files. Even though the file name (base_file) not monotonic,
# # I can order tham by the position in the list (i)
# base_files = sorted(base_files)
delta_file = delta_files[0]

for i, base_file in tqdm(enumerate(base_files)):

    # # Break in the middle to delete some files and save spae
    # if i >= 20:
    #     break
    print(f"\nProcess file {i}: {base_file}")

    state_dict = torch.load(base_file)
    file_name = f"pytorch_model-{i:02}.bin"
    for name, param in state_dict.items():
        if name not in delta_state_dict:
            for delta_file in delta_files:
                delta_state_dict = torch.load(delta_file)
                gc.collect()
                if name in delta_state_dict:
                    break
        if "embed_tokens" in name or "lm_head.weight" in name or "self_attn.rotary_emb.inv_freq" in name:
            state_dict[name] = delta_state_dict[name]
        else:
            # A terrible workaround in case the sizes of base and delta don't match
            delta_weights = delta_state_dict[name]
            if state_dict[name].size() != delta_weights.size():
                print(f'''\nSize of {name} in {base_file}: {state_dict[name].size()}
                    and in {delta_file}: {delta_weights.size()}''')
                delta_weights = delta_weights.\
                    narrow(0, 0, state_dict[name].shape[0]).narrow(1, 0, state_dict[name].shape[1])

            state_dict[name] += delta_weights
        weight_map[name] = file_name
        total_size += param.numel() * param.element_size()
        gc.collect()
    torch.save(state_dict, os.path.join(target_model_path, file_name))



Applying the delta


0it [00:00, ?it/s]


Process file 0: /content/tmp/base/pytorch_model-00006-of-00033.bin

Size of model.layers.5.self_attn.q_proj.weight in /content/tmp/base/pytorch_model-00006-of-00033.bin: torch.Size([4096, 4096])
                    and in /content/tmp/delta/pytorch_model-00004-of-00028.bin: torch.Size([5120, 5120])

Size of model.layers.5.self_attn.k_proj.weight in /content/tmp/base/pytorch_model-00006-of-00033.bin: torch.Size([4096, 4096])
                    and in /content/tmp/delta/pytorch_model-00004-of-00028.bin: torch.Size([5120, 5120])

Size of model.layers.5.self_attn.v_proj.weight in /content/tmp/base/pytorch_model-00006-of-00033.bin: torch.Size([4096, 4096])
                    and in /content/tmp/delta/pytorch_model-00004-of-00028.bin: torch.Size([5120, 5120])

Size of model.layers.5.self_attn.o_proj.weight in /content/tmp/base/pytorch_model-00006-of-00033.bin: torch.Size([4096, 4096])
                    and in /content/tmp/delta/pytorch_model-00004-of-00028.bin: torch.Size([5120, 5120])


0it [04:29, ?it/s]


Size of model.layers.5.input_layernorm.weight in /content/tmp/base/pytorch_model-00006-of-00033.bin: torch.Size([4096])
                    and in /content/tmp/delta/pytorch_model-00005-of-00028.bin: torch.Size([5120])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 16>:41                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: tuple index out of range

In [ ]:
with open(
    os.path.join(target_model_path, "pytorch_model.bin.index.json"), "w"
) as f:
    json.dump(
        {"weight_map": weight_map, "metadata": {"total_size": total_size}}, f
    )

print(f"Saving the target model to {target_model_path}")
delta_tokenizer.save_pretrained(target_model_path)
delta_config.save_pretrained(target_model_path)

# Debug

In [ ]:

!ls /root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce
# !ls -l tmp/base/
# !rm -rf /root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce/pytorch_model-0002*
# !ls -l /root/.cache/huggingface/hub/
# !rm -fr /root/.cache/huggingface/hub/models--decapoda-research--llama-7b-hf/
# !ls -l tmp/delta/

# !rm -f tmp/base/pytorch_model-0002*

ls: cannot access '/root/.cache/huggingface/hub/models--decapoda-research--llama-13b-hf/snapshots/438770a656712a5072229b62256521845d4de5ce': No such file or directory


In [ ]:
print(name)
print(delta_file)
delta_state_dict.keys()

model.layers.19.self_attn.q_proj.weight
/content/tmp/delta/pytorch_model-25.bin


dict_keys(['model.layers.18.mlp.up_proj.weight', 'model.layers.18.input_layernorm.weight', 'model.layers.18.post_attention_layernorm.weight', 'model.layers.19.self_attn.q_proj.weight', 'model.layers.19.self_attn.k_proj.weight', 'model.layers.19.self_attn.v_proj.weight', 'model.layers.19.self_attn.o_proj.weight', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.19.mlp.gate_proj.weight', 'model.layers.19.mlp.down_proj.weight', 'model.layers.19.mlp.up_proj.weight', 'model.layers.19.input_layernorm.weight', 'model.layers.19.post_attention_layernorm.weight', 'model.layers.20.self_attn.q_proj.weight', 'model.layers.20.self_attn.k_proj.weight', 'model.layers.20.self_attn.v_proj.weight', 'model.layers.20.self_attn.o_proj.weight', 'model.layers.20.self_attn.rotary_emb.inv_freq'])

dict_keys(['model.layers.19.self_attn.q_proj.weight', 'model.layers.19.self_attn.k_proj.weight', 'model.layers.19.self_attn.v_proj.weight', 'model.layers.19.self_attn.o_proj.weight', 'model.layers.19.mlp.gate_proj.weight', 'model.layers.19.mlp.down_proj.weight', 'model.layers.19.mlp.up_proj.weight', 'model.layers.19.input_layernorm.weight', 'model.layers.19.post_attention_layernorm.weight', 'model.layers.19.self_attn.rotary_emb.inv_freq'])

In [ ]:
tmp = torch.load('/content/tmp/delta/pytorch_model-24.bin')
tmp.keys()

dict_keys(['model.layers.33.mlp.up_proj.weight', 'model.layers.33.input_layernorm.weight', 'model.layers.33.post_attention_layernorm.weight', 'model.layers.34.self_attn.q_proj.weight', 'model.layers.34.self_attn.k_proj.weight', 'model.layers.34.self_attn.v_proj.weight', 'model.layers.34.self_attn.o_proj.weight', 'model.layers.34.self_attn.rotary_emb.inv_freq', 'model.layers.34.mlp.gate_proj.weight', 'model.layers.34.mlp.down_proj.weight', 'model.layers.34.mlp.up_proj.weight', 'model.layers.34.input_layernorm.weight', 'model.layers.34.post_attention_layernorm.weight', 'model.layers.35.self_attn.q_proj.weight', 'model.layers.35.self_attn.k_proj.weight', 'model.layers.35.self_attn.v_proj.weight', 'model.layers.35.self_attn.o_proj.weight', 'model.layers.35.self_attn.rotary_emb.inv_freq'])

In [ ]:
base_model_path
# !ls -l tmp/delta
!ls -l /root/.cache/huggingface/hub/models--decapoda-research--llama-7b-hf/blobs

total 13161796
-rw-r--r-- 1 root root 404770755 Jul 11 01:35 0d708f41d25ad5461d979337713757fabfe76c375b205b536bfe14d48556d5e0
-rw-r--r-- 1 root root       141 Jul 11 01:37 1aabb918c81c56bcb61ba76e6f93dc3ff601ee1a
-rw-r--r-- 1 root root 404770755 Jul 11 01:36 1b35993de04e54aad4a034da33ae2860220482c0be5f5c2b9555c13f969f6531
-rw-r--r-- 1 root root 404770755 Jul 11 01:37 2aef4f0698b0670d6e9c83ca103c3c8dff70e2f237065e2c7293f496933ead0e
-rw-r--r-- 1 root root 404770755 Jul 11 01:36 33d306be39a567f349b9e7bdc8b2543158607ca8834d8c71d32e1b648367ca94
-rw-r--r-- 1 root root 404770755 Jul 11 01:35 3cfe1d1093cbbb04d36b4fd09978e87077e41b4492b8cbba6bfff57c8dbd86dd
-rw-r--r-- 1 root root 404770755 Jul 11 01:37 3e746351f91d9d482c534620a441a7a66474fa9da57104b65f821fc513bad663
-rw-r--r-- 1 root root       427 Jul 11 01:34 437be862ab867ceb4b1a6e8d3c1187ade70c22c3
-rw-r--r-- 1 root root 404770755 Jul 11 01:35 6080c736b3136935f0d3fb9fc65b41008c65b6b7d5238e1b9c4780043f31b955
-rw-r--r-- 1 root root 404770755 J